In [ ]:
import dash
import dash_bootstrap_components as dbc
import pandas as pd
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

In [ ]:
# df = pd.read_parquet(
#     "/Users/jaeeunc/Desktop/MDS/Block5/DS551/project/data551-project/data/processed/wfp_preprocessed.parquet"
# )

df = pd.read_parquet('../data/processed/wfp_preprocessed.parquet')

countries = sorted(df["country"].unique())


def get_years(country):
    return sorted(df[df["country"] == country]["date"].dt.year.unique(), reverse=True)


default_country = countries[0]
default_year = get_years(default_country)[0]


def get_map(country, year, df=df):
    df_prep = df[df["country"] == country]
    df_grouped = (
        df_prep.groupby([df_prep["date"].dt.year, "admin2", "latitude", "longitude"])[
            "usdprice"
        ]
        .mean()
        .reset_index()
    )
    df_grouped["usdprice"] = df_grouped["usdprice"].apply(lambda x: round(x, 3))
    df_grouped_year = df_grouped[df_grouped["date"] == year]

    fig = px.scatter_map(
        df_grouped_year,
        lat="latitude",
        lon="longitude",
        size="usdprice",
        color="usdprice",
        color_continuous_scale="Oranges",
        hover_name="admin2",
        hover_data={"latitude": False, "longitude": False},
        zoom=5,
        height=600,
        width=600,
    )
    # "carto-positron", "carto-voyager"
    fig.update_layout(
        map_style="carto-positron",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        coloraxis_showscale=False,
    )
    fig.update_traces(
        hoverlabel=dict(
            bordercolor="rgba(0,0,0,0)",
            # bgcolor="rgba(255, 255, 255, 0.6)",
            font=dict(color="black"),
        )
    )

    return fig


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        html.H1(
            "Strategic Food Aid Planning: Regions & Prices at Risk",
            className="text-center text-dark mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        html.Label("Select Country", className="fw-bold"),
                        dcc.Dropdown(
                            id="country",
                            value=default_country,
                            options=[{"label": i, "value": i} for i in countries],
                            className="mb-3",
                        ),
                    ],
                    width=3,
                ),
                dbc.Col(
                    [
                        html.Label("Select Year", className="fw-bold"),
                        dcc.Dropdown(
                            id="year",
                            value=default_year,
                            options=[
                                {"label": i, "value": i}
                                for i in get_years(default_country)
                            ],
                            className="mb-3",
                        ),
                    ],
                    width=3,
                ),
            ],
            className="mb-4",
        ),
        dbc.Row(
            [
                dbc.Col(
                    [
                        dbc.Card(
                            [
                                dbc.CardHeader("Map Visualization"),
                                dbc.CardBody(
                                    [
                                        dcc.Graph(
                                            id="map-graph",
                                            figure=get_map(
                                                default_country, default_year
                                            ),
                                            style={
                                                "marginLeft": "30px",
                                                "marginRight": "50px",
                                            },
                                        )
                                    ]
                                ),
                            ],
                            className="md-5",
                        )
                    ],
                    width=6,
                ),
            ]
        ),
    ],
    fluid=True,
)


@app.callback(
    [Output("year", "options"), Output("year", "value")], Input("country", "value")
)
def update_year_options(selected_country):
    available_years = get_years(selected_country)
    return [{"label": i, "value": i} for i in available_years], available_years[0]


@app.callback(
    Output("map-graph", "figure"), [Input("country", "value"), Input("year", "value")]
)
def update_map(country, year):
    return get_map(country, year)


app.run_server(debug=True, open_browser=True)